# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [1]:
# Import tools and libraries
from dotenv import load_dotenv
from tavily import TavilyClient
import os
import json
import re
from pydantic import BaseModel
from openai import OpenAI
import chromadb
from lib.tooling import tool
# from langchain.tools import tool

from lib.agents import Agent

In [2]:
# Load environment variables
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [3]:

# ------------------------------
# Chroma setup
# ------------------------------
chroma_client = chromadb.PersistentClient(path="chromadb")

# ------------------------------
# RETRIEVE_GAME TOOL
# ------------------------------
@tool
def retrieve_game(query: str, n_results: int = 3):
    """
    Semantic search: Finds the most relevant games from the vector DB.

    Args:
        query (str): A question about the game industry.

    Returns:
        List[Dict]: Each result contains:
            - Platform (e.g., Game Boy, PS5, Xbox 360)
            - Name (name of the game)
            - YearOfRelease
            - Description
    """
    print("🔎 Retrieving from vector DB...")
    try:
        collection = chroma_client.get_collection("games_collection_new")

        results = collection.query(
            query_texts=query,
            n_results=n_results,
            include=["documents", "metadatas"]
        )

        metadatas = results.get("metadatas", [[]])[0]
        documents = results.get("documents", [[]])[0]

        output = []
        for meta, doc in zip(metadatas, documents):
            output.append({
                "Platform": meta.get("Platform"),
                "Name": meta.get("Name"),
                "YearOfRelease": meta.get("YearOfRelease"),
                "Description": meta.get("Description"),
                "document": doc
            })

        return {
            "data": output
        }

    except Exception as e:
        return [{"error": str(e)}]


#### Evaluate Retrieval Tool

In [4]:

class EvaluationReport(BaseModel):
    useful: bool
    description: str

@tool
def evaluate_retrieval(question: str, retrieved_docs: list[dict]):
    """
    LLM judge to evaluate whether retrieved docs are sufficient.
    """
    print("🧠 Evaluating retrieval...")

    prompt = f"""
You are an evaluation assistant. Your task is to decide whether the retrieved documents are sufficient to answer the user's question.

Question:
{question}

Retrieved Documents:
{json.dumps(retrieved_docs, indent=2)}

Return ONLY valid JSON in this format:

{{"useful": true, "description": "..."}}
"""

    client = OpenAI(api_key=OPENAI_API_KEY, base_url=os.getenv("OPENAI_BASE_URL"))

    resp = client.chat.completions.create(
        model=os.getenv("EVAL_MODEL", "gpt-4o-mini"),
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0
    )

    raw = resp.choices[0].message.content

    # --- JSON extraction ---
    try:
        parsed = json.loads(raw)
    except:
        match = re.search(r"\{[\s\S]*\}", raw)
        if not match:
            return {
                "useful": False,
                "description": f"Failed to extract JSON. Raw: {raw}"
            }
        parsed = json.loads(match.group(0))

    try:
        report = EvaluationReport.model_validate(parsed)
    except Exception as e:
        return {
            "useful": False,
            "description": f"Invalid JSON schema: {str(e)}. Raw: {parsed}"
        }

    return report.model_dump()



#### Game Web Search Tool

In [5]:

@tool
def game_web_search(question: str, max_results: int = 3) -> dict:
    """
    Uses Tavily client to search the web for a gaming-related question.
    Returns a formatted, human-readable summary of sources.
    """
    print("🌐 Searching the web...")
    tavily = TavilyClient(api_key=TAVILY_API_KEY)
    resp = tavily.search(
        query=question,
        include_answer=True,
        max_results=max_results
    )

    return {
        "answers": resp.get("results", []),
    }


### Agent

In [6]:
tools = [
    retrieve_game,
    evaluate_retrieval,
    game_web_search
]

uda_agent = Agent(
    model_name="gpt-4o-mini",
    tools=tools,
    instructions=(
        """
        You are a GameAgent, when a game-related question is asked, you MUST follow this pipeline:
        1. Retrieve relevant documents from the local game database using the retrieve_game tool.
        2. Evaluate the quality of the retrieved documents using the evaluate_retrieval tool.
        3. If the retrieved documents are sufficient, provide a concise answer based on them.
        4. If the documents are insufficient, search the web using the game_web_search tool. IF you use the `game_web_search` tool, then your FINAL ANSWER MUST follow this exact format:
         Based on web search results:
         - According to {title} ({url}): "{content}"
         - According to {title} ({url}): "{content}"

         The {title} MUST come from the web search result field "title".
         The {content} MUST come from the result field "content".
         The {url} MUST come from the field "url"
         """
    ))

In [7]:


queries = [
    "What is the genre of the game Super Mario 64?",
    "When was it released?",
    "Why is Minecraft so popular?",
]

for q in queries:
    print("\n========================================")
    print("QUESTION:", q)
    print("========================================\n")

    data = uda_agent.invoke(q, session_id='games_2')
    result = data.get_final_state()
    last_answer = next(
        (m.content for m in reversed(result["messages"])
         if m.role == "assistant" and m.content),
        None
    )
    print(last_answer)



QUESTION: What is the genre of the game Super Mario 64?

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
🔎 Retrieving from vector DB...


C:\Projects\building-agents\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
🧠 Evaluating retrieval...
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
The genre of the game Super Mario 64 is a 3D platformer. It was released in 1996 for the Nintendo 64 and is known for setting new standards in the genre.

QUESTION: When was it released?

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
Super Mario 64 was released in 1996.

QUESTION: Why is Minecraft so popular?

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
🌐 Searching the web...
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
Based on web search results:
- Acco

### (Optional) Advanced

In [8]:
import uuid
from datetime import datetime
from chromadb.utils import embedding_functions

# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes

embedding_fn = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")
long_term_memory = chroma_client.get_or_create_collection(name="long_term_memory", embedding_function=embedding_fn)


@tool
def store_memory(query: str, answer: str):
    """
    Store the final answer in long-term memory.
    Both query and answer are saved together so that future recall works.
    """
    print("🧠 Storing memory...")
    memory_id = str(uuid.uuid4())

    # Combine into one document (better for embedding & recall)
    memory_document = f"QUERY: {query}\nANSWER: {answer}"

    long_term_memory.add(
        documents=[memory_document],
        ids=[memory_id],
        metadatas=[{
            "timestamp": datetime.now().isoformat(),
            "query": query
        }]
    )
    return f"Memory saved for query: {query}"

@tool
def recall_relevant_memory(query: str, k: int = 1):
    """
    Retrieves the top-k most relevant memories for the agent.
    Returns a structured object including the source for traceability.
    """
    print("🧠 Recalling relevant memories...")

    results = long_term_memory.query(
        query_texts=[query],
        n_results=k
    )

    # No results case — still return a source tag
    if len(results["documents"]) == 0 or len(results["documents"][0]) == 0:
        return {
            "source": "long_term_memory",
            "found": False,
            "memories": [],
            "text": "No relevant memory found."
        }

    # Extract as list of strings
    memories = results["documents"][0]

    return {
        "source": "long_term_memory",
        "found": True,
        "memories": memories,
        "text": "\n".join([f"- {m}" for m in memories])
    }


In [9]:
store_memory("Where can I play the game Zula?", "Zula can be played on PC as a free-to-play online game.")
recall_relevant_memory("Where can I play the game Zula?")

🧠 Storing memory...
🧠 Recalling relevant memories...


{'source': 'long_term_memory',
 'found': True,
 'memories': ['QUERY: Where can I play the game Zula?\nANSWER: Zula can be played on PC as a free-to-play online game.'],
 'text': '- QUERY: Where can I play the game Zula?\nANSWER: Zula can be played on PC as a free-to-play online game.'}

In [13]:
uda_agent_with_memory = Agent(
    model_name="gpt-4o-mini",
    tools=tools + [store_memory, recall_relevant_memory],
    instructions=("""
You are a GameAgent with long-term memory.

IMPORTANT EXECUTION RULES:
- You MUST follow the pipeline exactly.
- You MUST NOT output any natural-language answer directly unless it is the final step where memory already exists.
- When memory must be stored, the answer MUST be returned inside the store_memory tool call.
- NEVER mix normal text with a tool call in the same message.

PIPELINE:

1. ALWAYS call recall_relevant_memory(query).
   - If memory is found (found=True):
       → Return the final user answer normally (no tool calls after this).
       → END.

2. If found=False:
       Call retrieve_game(query).

3. Then call evaluate_retrieval.
   - If "useful": true:
        → Create the final answer text.
        → Call store_memory with:
              { "query": <user_query>, "answer": <final_answer> }
        (The tool call IS your final message. Do NOT output the final answer outside the tool.)
        → END.
   - If "useful": false:
         → Call game_web_search(query)


5. After receiving search results from game_web_search:
       → Build the formatted answer string exactly like:
            Based on web search results:
            - According to {title} ({url}): "{content}"
            - According to {title} ({url}): "{content}"

         The {title} MUST come from the web search result field "title".
         The {content} MUST come from the result field "content".
         The {url} MUST come from the field "url"

       → Then call store_memory with:
              { "query": <user_query>, "answer": <final_answer> }
       (Again: ONLY output the tool call, do NOT output the final answer outside the tool.)
       → END.

        """))


In [14]:
queries = [
    "What is the genre of the game Halo Infinite",
    "When was it released?",
    "On which platforms can I play The Witcher 3?",
    "What made FarmVille so successful?"
]

for q in queries:
    print("\n========================================")
    print("QUESTION:", q)
    print("========================================\n")

    data = uda_agent_with_memory.invoke(q)
    messages = data.get_final_state()["messages"]
    result = []
    for msg in reversed(messages):
        if msg.role == "assistant" and msg.content:
            result = msg.content
            break
    print(result)





QUESTION: What is the genre of the game Halo Infinite

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
🧠 Recalling relevant memories...
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
Halo Infinite is a first-person shooter game.

QUESTION: When was it released?

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
🧠 Recalling relevant memories...
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
Halo Infinite was released in 2021.

QUESTION: On which platforms can I play The Witcher 3?

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
🧠 Recalling relevant memories...
[StateMachine] Executing step: t